In [ ]:
# python lib
import os
import shutil
import sys
import subprocess
import time
original_directory = os.getcwd()
# import alolib
import copy

# local import
os.chdir(os.path.abspath(os.path.join('..')))
from common import Logger, print_color, find_matching_strings, asset_info, extract_requirements_txt, check_install_requirements

In [ ]:
req_list = extract_requirements_txt("master")
master_req = {"master": req_list}

check_install_requirements(master_req)

In [ ]:
from alolib.asset import Asset
from alolib.exception import print_color

In [ ]:
# !make clean

In [ ]:
# !make del

In [ ]:
os.chdir(os.path.abspath(os.path.join(original_directory)))

In [ ]:
# 현재 위치 확인

PROJECT_HOME = os.path.dirname(os.getcwd())
# experimental plan yaml의 위치
EXP_PLAN = PROJECT_HOME + "/config/experimental_plan.yaml"
# asset 코드들의 위치
SCRIPT_HOME = PROJECT_HOME + "/assets/"

envs = {}
envs['project_home'] = PROJECT_HOME + "/"

SUPPORT_TYPE = ['memory', 'file']

In [ ]:
req_list = extract_requirements_txt("master")
master_req = {"master": req_list}
check_install_requirements(master_req)

In [ ]:
# asset init
asset = Asset(envs=envs, argv=0, version=0.1)

In [ ]:
# configure setting
asset.get_yaml(EXP_PLAN)
external_path = asset.get_external_path()
external_path_permission = asset.get_external_path_permission()
pipelines_list = asset.get_pipeline()
user_parameters = asset.get_user_parameters()
controls = asset.get_control()
artifacts = asset.set_artifacts()

In [ ]:
# 외부 데이터 다운로드 (input 폴더에)
asset.fetch_data(external_path, external_path_permission)

In [ ]:
pipe_mode = 'train_pipeline'

In [ ]:
####################### ALO master requirements 리스트업 #######################
# ALO master requirements 는 최우선 순위로 설치 > 만약 ALO master requirements는 aiplib v2.1인데 slave 제작자가 aiplib v2.2로 명시해놨으면 2.1이 우선 
requirements_dict = dict() 
requirements_dict['master'] =  extract_requirements_txt(step_name = 'master')
####################### Slave Asset 설치 및 Slave requirements 리스트업 #######################

# setup asset (asset을 git clone (or local) 및 requirements 설치)
get_asset_source = controls["get_asset_source"]  # once, every
for step, asset_config in enumerate(pipelines_list[pipe_mode]):
    # self.asset.setup_asset 기능 :
    # local or git pull 결정 및 scripts 폴더 내에 위치시킴 
    asset.setup_asset(asset_config, get_asset_source)
    requirements_dict[asset_config['step']] = asset_config['source']['requirements']
    # local 모드일 땐 이번 step(=asset)의 종속 package들이 내 환경에 깔려있는 지 항상 체크 후 없으면 설치 
    # git 모드일 땐 every이거나 once면서 첫 실행 시에만 requirements 설치 

####################### Master & Slave requirements 설치 #######################
# 이미 asset (step) 폴더들은 input 폴더에 다 setup된 상태 
# 각 asset의 yaml에 직접 작성 된 패키지들 + asset 내의 requirements.txt를 참고하여 쭉 리스트업 
# asset 간 중복 패키지 존재 시 먼저 실행되는 pipeline, asset에 대해 우선순위 부여하여 설치되게끔   
# 패키지 설치 중에 진행중인 asset 단계 표시 및 총 설치 중 몇번 째 설치인지 표시 > pipeline 별로는 별도로 진행 
asset.check_install_requirements(requirements_dict) 

In [ ]:
check_asset_source = controls["get_asset_source"]

In [ ]:
def run(step, args, pipelines, data, pipe_val):
    asset_config = pipelines_list[pipelines][step]
    asset.setup_asset(asset_config, check_asset_source)
    
    asset_info(pipelines, asset_config['step'])
    
    # scripts 폴더에 있는 내용을 가져와 import 한다
    _path = SCRIPT_HOME + asset_config['step'] + "/"
    _file = "asset_" + asset_config['step']
    user_asset = asset.import_asset(_path, _file)
    
    if controls['interface_mode'] in SUPPORT_TYPE:
        # 첫 동작시에는 초기화하여 사용 
        if step == 0:
            data = 0
            pipe_val = {}
        else:
            if controls['interface_mode'] == 'memory':
                pass
            elif controls['interface_mode'] == 'file':
                data, pipe_val = asset.get_toss(pipelines, envs) # file interface
    else:
        return ValueError("only file and memory")
    
    envs = {}
    envs['project_home'] = PROJECT_HOME
    envs['pipeline'] = pipelines
    envs['step'] = user_parameters[pipelines][step]['step']
    envs['artifacts'] = artifacts
    
    ua = user_asset(envs, args, data, pipe_val)
    data, pipe_val = ua.run()
    
    # self.asset.save_file(data)

    sys.path = [item for item in sys.path if envs['step'] not in item]

    if controls['interface_mode'] == 'file':
        asset.toss(data, pipe_val, pipelines, envs)
    else:
        return data, pipe_val

In [ ]:
data = 0
pipe_val = 0

In [ ]:
pipelines = 'train_pipeline'

In [ ]:
step = 0
args = user_parameters[pipelines][step]['args'][0]
if controls['interface_mode'] == 'file':
    run(step, pipelines, data, pipe_val)
else:
    data, config = run(step, args, pipelines, data, pipe_val)

In [ ]:
origin_data = copy.copy(data)

In [ ]:
origin_config = copy.copy(config)

In [ ]:
# read data from file
# get_toss(pipelines, envs)
# pipelines: 현재 동작중인 파이프라인 이름
# 현재 동작중인 환경 설정
if controls['interface_mode'] == 'file':
    step = 0
    # step: 현재 동작중인 asset number
    def make_envs(step):
        envs = {}
        envs['pipeline'] = pipelines
        envs['step'] = user_parameters[pipelines][step]['step']
        envs['artifacts'] = artifacts
        return envs
    data, pipe_val = asset.get_toss(pipelines, make_envs(step))

In [ ]:
# 확인을 위한 원본 데이터 백업
data = copy.copy(origin_data)
config = copy.copy(origin_config)

In [ ]:
step = 1
args = user_parameters[pipelines][step]['args'][0]
# args['handling_missing'] = 'dropna'
data, config = run(step, args, pipelines, data, config)

In [ ]:
data

In [ ]:
step = 1
args = user_parameters[pipelines][step]['args'][0]
args['handling_missing'] = 'dropna'
data, config = run(step, args, pipelines, data, config)
data

In [ ]:
step = 1
data, pipe_val = run(step, pipelines, data_origin, pipe_val_origin)

In [ ]:
step = 1
data, pipe_val = run(step, pipelines, data, pipe_val)

In [ ]:
pipe_val_origin

In [ ]:
step = 2
data, pipe_val = run(step, pipelines, data, pipe_val)

In [ ]:
# 넘어가면 실행이 안됨
step = 3
data, pipe_val = run(step, pipelines, data, pipe_val)

In [ ]:
data, pipe_val = run(0, 'inference_pipeline', data, pipe_val)

In [ ]:
run(1, 'inference_pipeline', data, pipe_val)

In [ ]:
#위에 생성된 data와 config를 이용해 다음 step을 진행

In [ ]:
# input asset이 아닌 경우 data와 전역 변수 config가 필요하다
# data의 경우 현재 step에서 사용할 data를 넣는다고 생각하면 된다
# import pandas as pd
# data = pd.read_csv("/home/jovyan/repo/alov2/train_data/bolt_train_data.csv")